# Unit 6 Exercise


### Rock–Paper–Scissors (Kaggle)
```
Rock-Paper-Scissors/
  train/
    rock/ paper/ scissors/
  test/
    rock/ paper/ scissors/
  validation/
```

In [1]:
# Imports
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers

# Reproducibility (optional)
tf.random.set_seed(42)
np.random.seed(42)

# Create models folder
os.makedirs("models", exist_ok=True)

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


# PARTS 1–2 — Muffin vs Chihuahua (Improved CNN)

In [2]:
# Muffin vs Chihuahua paths
train_dir = "train_1"
test_dir = "test_1"

# Image parameters
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

print("train_dir exists?", os.path.isdir(train_dir))
print("test_dir exists?", os.path.isdir(test_dir))
print("run_1 exists?", os.path.isdir("run_1"))

if os.path.isdir(train_dir):
    print("Train class folders:", sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]))
if os.path.isdir(test_dir):
    print("Test class folders:", sorted([d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]))

train_dir exists? True
test_dir exists? True
run_1 exists? True
Train class folders: ['chihuahua', 'muffin']
Test class folders: ['chihuahua', 'muffin']


In [3]:
# Data generators (with augmentation + validation split)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

print("Class indices:", train_generator.class_indices)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Class indices: {'chihuahua': 0, 'muffin': 1}


In [4]:
# Improved CNN model (required: dropout + regularization)
l2 = regularizers.l2(1e-4)

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model = models.Sequential([
    layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=l2),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Train the Muffin vs Chihuahua model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 58s 472ms/step - accuracy: 0.6122 - loss: 0.7427 - val_accuracy: 0.7831 - val_loss: 0.5729
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 289ms/step - accuracy: 0.7376 - loss: 0.5702 - val_accuracy: 0.7354 - val_loss: 0.5568
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 284ms/step - accuracy: 0.7746 - loss: 0.5194 - val_accuracy: 0.8360 - val_loss: 0.4653
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 282ms/step - accuracy: 0.8155 - loss: 0.4609 - val_accuracy: 0.8212 - val_loss: 0.4468
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 34s 284ms/step - accuracy: 0.8276 - loss: 0.4309 - val_accuracy: 0.8899 - val_loss: 0.3494
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 36s 299ms/step - accuracy: 0.8440 - loss: 0.4038 - val_accuracy: 0.8772 - val_loss: 0.3665
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 35s 296ms/step - accuracy: 0.8453 - loss: 0.3919 - val_accuracy: 0.8868 - val_loss: 0.3327
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 35s 290ms/step - accuracy: 0.8508 - loss: 0

In [6]:
# Evaluate the model (Accuracy)
test_loss, test_acc = model.evaluate(test_generator)
print(f"Muffin vs Chihuahua Test Accuracy: {test_acc}")

if 'val_accuracy' in history.history:
    print(f"Muffin vs Chihuahua Final Val Accuracy: {history.history['val_accuracy'][-1]}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 17s 468ms/step - accuracy: 0.8277 - loss: 0.3969
Muffin vs Chihuahua Test Accuracy: 0.8277027010917664
Muffin vs Chihuahua Final Val Accuracy: 0.8878306746482849


In [7]:
# Save the improved model with required filename
improved_path = "models/exercise_6_trained_model_improved.h5"
model.save(improved_path)
print("Saved:", improved_path)

Saved: models/exercise_6_trained_model_improved.h5


In [9]:
# Inference: Run 1 & Run 2 prediction and confidence
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_obj):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = float(model_obj.predict(img_array, verbose=0)[0, 0])

    # Keep it simple but clear:
    # - 'pred' is the model probability for the positive class (based on class_indices mapping)
    # - We also show a "predicted label confidence" so the printed confidence matches the label.
    predicted_label = "Chihuahua" if pred >= 0.5 else "Muffin"
    label_conf = pred if predicted_label == "Chihuahua" else (1 - pred)

    print(f"Image: {img_path}")
    print(f"Raw sigmoid output: {pred:.4f}")
    print(f"Prediction: {predicted_label} (confidence: {label_conf:.4f})")
    print("-")

# Example usage (make sure these files exist)
predict_image("run_1/run_1.jpg", model)
predict_image("run_1/run_2.jpg", model)

Image: run_1/run_1.jpg
Raw sigmoid output: 0.6243
Prediction: Chihuahua (confidence: 0.6243)
-
Image: run_1/run_2.jpg
Raw sigmoid output: 0.9800
Prediction: Chihuahua (confidence: 0.9800)
-


# PART 3 — Custom Dataset (Rock–Paper–Scissors)
Dataset: Rock–Paper–Scissors (Kaggle)

This dataset is **3-class**, so we must use:
- `class_mode='categorical'`
- Output: `Dense(3, softmax)`
- Loss: `categorical_crossentropy`

In [10]:
# Rock-Paper-Scissors paths
RPS_ROOT = "Rock-Paper-Scissors"
rps_train_dir = os.path.join(RPS_ROOT, "train")
rps_test_dir  = os.path.join(RPS_ROOT, "test")

print("RPS train exists?", os.path.isdir(rps_train_dir))
print("RPS test exists?", os.path.isdir(rps_test_dir))

IMG_SIZE_RPS = (128, 128)
BATCH_SIZE_RPS = 32

RPS train exists? True
RPS test exists? True


In [11]:
# RPS generators (validation split from train/)
rps_train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

rps_test_datagen = ImageDataGenerator(rescale=1./255)

rps_train_generator = rps_train_datagen.flow_from_directory(
    rps_train_dir,
    target_size=IMG_SIZE_RPS,
    batch_size=BATCH_SIZE_RPS,
    class_mode='categorical',
    subset='training'
)

rps_val_generator = rps_train_datagen.flow_from_directory(
    rps_train_dir,
    target_size=IMG_SIZE_RPS,
    batch_size=BATCH_SIZE_RPS,
    class_mode='categorical',
    subset='validation'
)

rps_test_generator = rps_test_datagen.flow_from_directory(
    rps_test_dir,
    target_size=IMG_SIZE_RPS,
    batch_size=BATCH_SIZE_RPS,
    class_mode='categorical',
    shuffle=False
)

print("RPS class indices:", rps_train_generator.class_indices)

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Found 372 images belonging to 3 classes.
RPS class indices: {'paper': 0, 'rock': 1, 'scissors': 2}


In [12]:
# Simple CNN for RPS (3-class)
rps_model = models.Sequential([
    layers.Input(shape=(IMG_SIZE_RPS[0], IMG_SIZE_RPS[1], 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

rps_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
rps_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# Train RPS custom dataset model
rps_history = rps_model.fit(
    rps_train_generator,
    epochs=10,
    validation_data=rps_val_generator
)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 35s 543ms/step - accuracy: 0.8199 - loss: 0.4024 - val_accuracy: 0.6845 - val_loss: 0.7003
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 139ms/step - accuracy: 0.9985 - loss: 0.0044 - val_accuracy: 0.6885 - val_loss: 1.4435
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 141ms/step - accuracy: 1.0000 - loss: 3.5130e-04 - val_accuracy: 0.7044 - val_loss: 1.3235
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 138ms/step - accuracy: 1.0000 - loss: 1.0364e-04 - val_accuracy: 0.7123 - val_loss: 1.0942
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 140ms/step - accuracy: 1.0000 - loss: 5.6590e-05 - val_accuracy: 0.7024 - val_loss: 1.3402
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 140ms/step - accuracy: 1.0000 - loss: 3.1326e-05 - val_accuracy: 0.7063 - val_loss: 1.2333
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 136ms/step - accuracy: 1.0000 - loss: 1.8224e-05 - val_accuracy: 0.7044 - val_loss: 1.3716
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 138ms/step - accuracy: 1.0000 - loss: 1.20

In [14]:
# Evaluate RPS model
rps_test_loss, rps_test_acc = rps_model.evaluate(rps_test_generator)
print("RPS Custom CNN Test Accuracy:", rps_test_acc)

if 'val_accuracy' in rps_history.history:
    print("RPS Custom CNN Final Val Accuracy:", rps_history.history['val_accuracy'][-1])

12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 344ms/step - accuracy: 0.8333 - loss: 0.9968
RPS Custom CNN Test Accuracy: 0.8333333134651184
RPS Custom CNN Final Val Accuracy: 0.7103174328804016


In [15]:
# Save custom dataset model (replace lastname when ready)
custom_path = "models/exercise_6_custom.pastolero.h5"
rps_model.save(custom_path)
print("Saved:", custom_path)

Saved: models/exercise_6_custom.pastolero.h5


# PART 4 — Transfer Learning (VGG16) on RPS
We use VGG16 with ImageNet weights and a small classifier head.

- Input size: 224×224 (standard for VGG16)

In [16]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

IMG_SIZE_TL = (224, 224)
BATCH_SIZE_TL = 32

tl_train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

tl_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

tl_train_generator = tl_train_datagen.flow_from_directory(
    rps_train_dir,
    target_size=IMG_SIZE_TL,
    batch_size=BATCH_SIZE_TL,
    class_mode='categorical',
    subset='training'
)

tl_val_generator = tl_train_datagen.flow_from_directory(
    rps_train_dir,
    target_size=IMG_SIZE_TL,
    batch_size=BATCH_SIZE_TL,
    class_mode='categorical',
    subset='validation'
)

tl_test_generator = tl_test_datagen.flow_from_directory(
    rps_test_dir,
    target_size=IMG_SIZE_TL,
    batch_size=BATCH_SIZE_TL,
    class_mode='categorical',
    shuffle=False
)

print("TL class indices:", tl_train_generator.class_indices)

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Found 372 images belonging to 3 classes.
TL class indices: {'paper': 0, 'rock': 1, 'scissors': 2}


In [17]:
# Build VGG16 transfer learning model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE_TL[0], IMG_SIZE_TL[1], 3))
base_model.trainable = False

tl_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

tl_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
tl_model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,780,739 (56.38 MB)

 Trainable params: 66,051 (258.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [18]:
# Train transfer learning model (kept small for simplicity)
tl_history = tl_model.fit(
    tl_train_generator,
    epochs=5,
    validation_data=tl_val_generator
)

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.9211 - loss: 0.2121 - val_accuracy: 0.9802 - val_loss: 0.0870
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 1.0000 - loss: 0.0068 - val_accuracy: 0.9881 - val_loss: 0.0577
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 0.9881 - val_loss: 0.0468
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.9921 - val_loss: 0.0338
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 1.0000 - loss: 8.8358e-04 - val_accuracy: 0.9901 - val_loss: 0.0315


In [19]:
# Evaluate transfer learning model
tl_test_loss, tl_test_acc = tl_model.evaluate(tl_test_generator)
print("VGG16 Transfer Learning Test Accuracy:", tl_test_acc)

if 'val_accuracy' in tl_history.history:
    print("VGG16 Final Val Accuracy:", tl_history.history['val_accuracy'][-1])

12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8737 - loss: 0.3831
VGG16 Transfer Learning Test Accuracy: 0.8736559152603149
VGG16 Final Val Accuracy: 0.9900793433189392


In [20]:
# Save transfer learning model
tl_path = "models/exercise_6_transfer_vgg16_rps.h5"
tl_model.save(tl_path)
print("Saved:", tl_path)

Saved: models/exercise_6_transfer_vgg16_rps.h5
